# Testing openai cho việc giải toán.

In [1]:
from openai import OpenAI
import os
import base64
import requests
from dotenv import load_dotenv
load_dotenv('../.env')

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
MODEL_NAME = "gpt-4o-mini"

In [2]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


In [3]:
image_url = "./toan-trac-nghiem.png"
image_base64 = encode_image(image_url)
print(image_base64[:100])

iVBORw0KGgoAAAANSUhEUgAABJ4AAACgCAYAAABNLKYkAAAMPmlDQ1BJQ0MgUHJvZmlsZQAASImVVwdYU8kWnltSIbQAAlJCb4KI


In [21]:
system_message = "Bạn là một giáo viên toán cấp 3, đang hướng dẫn học sinh thi tốt nghiệp đại học."
response = client.chat.completions.create(
  model=MODEL_NAME,
  messages=[
    {"role": "system", "content": system_message},
    {
        "role": "user",
        "content": [
        {
            "type": "text",
            "text": "Chọn đáp án đúng và đưa ra lời giải thích",
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{image_base64}"
            },
        },
      ]
    },
  ],
  max_tokens=300,
)

In [26]:
print(response.choices[0].message.content)

Để giải bài toán này, ta sử dụng định lý logarithm và các tính chất của logarit:

Ta có:
\[
\ln(7a) - \ln(3a)
\]

Áp dụng tính chất của logarit:
\[
\ln(a) - \ln(b) = \ln\left(\frac{a}{b}\right)
\]
Vậy ta sẽ có:
\[
\ln(7a) - \ln(3a) = \ln\left(\frac{7a}{3a}\right)
\]

Rút gọn biểu thức bên trong:
\[
\frac{7a}{3a} = \frac{7}{3}
\]

Như vậy, biểu thức sẽ trở thành:
\[
\ln\left(\frac{7}{3}\right)
\]

Do đó, đáp án đúng là:
\[
\text{C. } \ln\left(\frac{7}{3}\right)
\]


## Cách sử dụng request API.

In [18]:


# OpenAI API Key
api_key = os.getenv('OPENAI_API_KEY')
system_message = "Bạn là một giáo viên toán cấp 3, đang hướng dẫn học sinh thi tốt nghiệp đại học."

# Getting the base64 string
base64_image = encode_image(image_url)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o-mini",
  "messages": [
    {
      "role": "system",
      "content": system_message
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Chọn đáp án đúng và đưa ra lời giải thích"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{image_base64}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9zfkvyMEqBwpbgWgMI5EZko1iGfvp', 'object': 'chat.completion', 'created': 1724485545, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Để giải bài toán này, ta sử dụng tính chất của logarithm tự nhiên.\n\nTa có:\n\n\\[\n\\ln(7a) - \\ln(3a) = \\ln\\left(\\frac{7a}{3a}\\right)\n\\]\n\nKhi đó, \\(\\ln(a)\\) trong tử và mẫu sẽ bị hủy:\n\n\\[\n\\ln\\left(\\frac{7a}{3a}\\right) = \\ln\\left(\\frac{7}{3}\\right)\n\\]\n\nVậy, ta sẽ được:\n\n\\[\n\\ln(7a) - \\ln(3a) = \\ln\\left(\\frac{7}{3}\\right)\n\\]\n\nDo đó, đáp án đúng là:\n\n**C. \\(\\ln\\left(\\frac{7}{3}\\right)\\)**.', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 19877, 'completion_tokens': 171, 'total_tokens': 20048}, 'system_fingerprint': 'fp_507c9469a1'}


## Sử dụng `beta.chat.completions.parse` để tạo response hướng dẫn từng bước cho người dùng.

In [5]:
from pydantic import BaseModel
import openai

class Step(BaseModel):
    explanation: str
    output: str

class MathResponse(BaseModel):
    steps: list[Step]
    final_answer: str


system_message = "Bạn là một gia sư toán hữu ích. Hướng dẫn người dùng từng bước giải quyết"

completion = client.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": system_message},
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_base64}",
                    },
                },
            ],
        },
    ],
    response_format=MathResponse,
    max_tokens=300
)
math_response = completion.choices[0].message
if math_response.parsed:
    print(math_response.parsed)
elif math_response.refusal:
    # handle refusal
    print(math_response.refusal)


steps=[Step(explanation='Sử dụng tính chất của logarit: \\ln(a) - \\ln(b) = \\ln\\left(\\frac{a}{b}\\right) để biểu thức có thể đơn giản hóa.', output='\\ln(7a) - \\ln(3a) = \\ln\\left(\\frac{7a}{3a}\\right) = \\ln\\left(\\frac{7}{3}\\right) '), Step(explanation='Giá trị biểu thức \\ln(7a) - \\ln(3a) sau khi rút gọn được thành \\ln\\left(\\frac{7}{3}\\right).', output='\\ln\\left(\\frac{7}{3}\\right) '), Step(explanation='So sánh kết quả với các phương án đã cho.', output='Kết quả là \\ln\\left(\\frac{7}{3}\\right) và phương án C là đúng.')] final_answer='C. \\ln \\left( \\frac{7}{3} \\right)'
